# Homework 03. 

#### MSCA 32013 Optimization and Simulation Methods. University of Chicago.

#### Group 7. Ben Ossyra, Peter Pezon, Yawen Zhang

## Q1. Arbitrage trade: hypothetical bid/ask quotes for four major currencies at a given date/time.

From Questions 2 from "Homework 03 - Linear Programming and Simplex Method."

The table below presents hypothetical bid/ask quotes for four major currencies at a given date/time:

Pair   | Bid   | Ask
-------|-------|-------
EUR/USD|0.87060|0.87063
GBP/USD|1.42790|1.42796
USD/JPY|133.330|133.333
EUR/GBP|0.60970|0.60972
EUR/JPY|116.140|116.144
GBP/JPY|190.480|190.476

The first currency in a currency pair is called the base currency. The bid and the ask are always quoted in terms of the base currency. For example, if you are buying on EUR/USD trade, then you are effectively receiving EUR in exchange for USD at the ask quote.

You were given $USD 1$ to attempt an arbitrage trade, assume no transaction costs.

---

References

[1] "Lecture 5. Foreign-Currency Trading." Decision Models. Columbia Business School. https://www.meiss.com/columbia/en/teaching/1999/fall/B6015/download/lecture05/lec5.pdf


### A. Write down a linear programming problem to buy and sell quantities of the base currencies and arbitrage/profit of at most USD  10k.

A *spot currency transaction* is an agreement to buy some amount of one currency using another currency. 

Filled out a cross-currency table for today's spot rates. Columns are indexed by base currency, rows are indexed by exchange currency. Cross-currency spot rates are calculated using the bid/ask quotes.

|       | To: | USD     | EUR     | GBP     | JPY     | 
| ----- | --- | ------- | ------- | ------- | --------|
| From: | USD |         | 0.87063 | 1.42790 | 0.00750 |
|       | EUR | 1.14859 |         | 1.64010 | 0.00861 |
|       | GBP | 0.70030 | 0.60972 |         | 0.00525 | 
|       | JPY | 133.333 | 116.140 | 190.480 |         | 

**Figure 1.** Today's Cross-Currency Spot Rates.

*Arbitrage* is a set of spot currency transactions that creates positive wealth but does not require any funds to initiate, i.e., it is a "money pump." The arbitrage could involve more than two currencies. if such opportunities exist, it is necessary to be able to identify them and act quickly. 

Problem Statement: Can a decision model be formulated to detect arbitrage opportunities in the spot currency market. Assume no transaction costs. 

What needs to be decided? A set of spot currency transactions.

What is the objective? Maximize the final net amount of $USD$. 

What are the constraints? How many constraints?

* The final net amount of each currency must be nonnegative. The total amount of all currencies converted into a base currency should be greater than the currency converted in other currencies.
* Nonnegative transactions only
* There should be one constraint for each currency. Therefore, there are 4 constraints.

*Indices*: Let $i = 1,...,4$ represent the currencies $USD$, $EUR$, $GBP$, and $JPY$, respectively.

*Decision variables*: Let $x_{i,j} = $ amount of currency $i$ to be converted into currency $j$ (measured in units of curency $i$) for $i = 1, ..., 4, j = 1, ..., 4$, and $i \neq j$. Therefore, there are 9 decision variables.

The "total out" of currency $k$ represents the total amount of currency $k$ converted to other currencies, measured in currency $k$. For example, for $USD$ this can be written as $0.87063 x_{2,1} + 1.42790 x_{3,1} + 0.00750 x_{4,1}$.

The "total in" to currency $k$ represents the total amount of other currencies converted into currency $i$, measured in currency $k$. For example, for $USD$ this can be written as $x_{1,2} + x_{1,3} + x_{1,4}$.

Final net to currency $k$ represents the final or net amount of currency $k$. For example, for $USD$ this can be written as $0.87063 x_{2,1} + 1.42790 x_{3,1} + 0.00750 x_{4,1} - (x_{1,2} + x_{1,3} + x_{1,4})$.

Let $f_{k}$ represent the final amount of currency $k$ (measured in units of currency $k$) for $k = 1, ..., 4$. That is, $f_{k}$ is the total converted into currency $k$ minus the total converted out of currency $k$. 

Finally, we are given $USD 1$ to attempt an arbitrage trade. We start with no other currencies. Let $I$ be the initial investment for each currency $i$. Let $I = [1, 0, 0, 0]$.

We set the currency $USD$ at index $i = 1$ because that is the currency of interest and the starting base currency. Therefore, we can say that we are maximizing $f_{1}$, or the final amount of currency measured in units of $USD$. 

Therefore the arbitrage linear programming model is:

$$ \max{f_{1}} $$
subject to: 

The final net amount $f_{k}$ definitions:

$$ f_{1} = 0.87063 x_{2,1} + 1.42790 x_{3,1} + 0.00750 x_{4,1} - (x_{1,2} + x_{1,3} + x_{1,4}) $$
$$ f_{2} = 1.14859 x_{1,2} + 1.64010 x_{3,2} + 0.00861 x_{4,2} - (x_{2,1} + x_{2,3} + x_{2,4}) $$
$$ f_{3} = 0.70030 x_{1,3} + 0.60972 x_{2,3} + 0.00525 x_{4,3} - (x_{3,1} + x_{3,2} + x_{3,4}) $$
$$ f_{4} = 133.333 x_{1,4} + 116.140 x_{2,4} + 190.480 x_{3,4} - (x_{4,1} + x_{4,2} + x_{4,3}) $$

Nonnegative boundary for all variables:

$ x_{i,j} \geq 0 $ for all $ i = 1, ..., 4, j = 1, ..., 4, i \neq j $

$ f_{k} \geq 0 $ for all $ k = 0, ..., 4 $

Bound on total arbitrage maximum profit of \$10,000. To ensure a profit of at most \$10,000, we constrain D to \$10,001 (the profit plus the dollar we start with)

$ f_{1} \leq 10001 $

### B. Solve for the buy/sell quantities and the arbitrage/profit amount.

In [3]:
using JuMP, GLPK, LinearAlgebra

m = Model(GLPK.Optimizer)

# Specify cross-currency spot rate matrix:
SR = [  0               0.87063         1.42790         (1/133.333) ;
        (1/0.87063)     0               (1/0.60972)     (1/116.144) ;
        (1/1.42796)     0.60972         0               (1/190.476) ;
        133.333         116.140         190.480         0           ]

# Initial investment:
I = [ 1; 0; 0; 0 ]

# Declare variables and non-negative boundaries for all variables:
@variable(m, x[i = 1:4, j = 1:4] .>= 0)
@variable(m, f[k = 1:4] .>= 0)

# Add final net amount f_k constraints (including initial investment I_i)
for i = 1:4
    @constraint(m, I[i] + dot(SR[i,:], x[:,i]) - sum(x[i,:]) == f[i])
end

# Add bound on total arbitrage (avoid infinite money):
@constraint(m, f[1] <= 10001)

# Set the objective:
@objective(m, Max, f[1])

# Print the model:
print(m)

# Solve the optimization problem:
JuMP.optimize!(m)

# Print the objective value:
println("Objective value: ", JuMP.objective_value(m))

# Print the optimal solutions obtained
println("f = ", JuMP.value.(f))
println("x = ", JuMP.value.(x))

Objective value: 10001.0
f = [10001.0, 0.0, 0.0, 0.0]
x = [0.0 2.0622362181625046e7 0.0 0.0; 0.0 0.0 0.0 2.3686712129865784e7; 0.0 0.0 0.0 0.0; 2.7509747467626123e9 0.0 0.0 0.0]


### C. Interpret the results and develop a story of the trade executions and the resulting profit.

The optimal solution uses three currencies, $USD$, $GBP$, and $JPY$. The indicated trades produce nonnegative amounts of all currencies and a positive amount of $USD$.

Starting with an initial investment of $1 USD$, it is exchanged for $EUR$.  

In [4]:
EUR = 1 * (1/0.87063) # (USD -> EUR)
print(EUR)

1.148593547201452

Then you exchange your 1.1486 EUR for JPY  

In [5]:
JPY = EUR * 116.140 # (EUR -> JPY)
print(JPY)

133.39765457197663

Then you exchange your 133.3976 JPY back to USD

In [6]:
USD = JPY * (1/133.333) # (JPY -> USD)
print(USD)

1.000484910502101

You have successfully converted 1 USD into ~1.0005 USD. So your abitrage profit is ~0.0005 USD. If you scale this by 20,622,362.18, you will make a profit of $10,000

In [15]:
20622362.181625046 * (USD-1)

10000.0

## Q2. Fuel production.

A refinery produces a high grade (H) and a low grade (L) fuel from five raw stocks with different octane rating. The price and the rating threshold for the two fuel types
are presented in the table below:

| Fuel | Rating | Price ($/B) |
| ---- | ------ | ----------- |
| High |	93	| 37.5        |
| Low  |	85	| 28.5        |

The rating, price, and quantity available of each raw material are presented below.

| Stock | Rating | Price | Quantity |
| ----- | ------ | ----- | -------- |
|  1    |	70	 |	9.0	 | 2000     |
|  2    |	80	 |	12.5 | 4000     |
|  3    |	85	 |	12.5 | 4000     |
|  4    |	90	 |	27.5 | 5000     |
|  5    |	99	 |	27.5 | 3000     |

The refinery can sell the high and low fuels and any unused raw materials. The goal is to maximize revenue. Find the optimal blend for the high grade and low grade fuel.

Problem statement: what is the right combination of raw stock to blend to create low and high grade fuel, and what quantity of raw stock, low grade and high grade fuels will maximize profit?

Our objective is to maximize profits, defined as quantity times price. Therefore we want to adjust the quantities of high-grade fuel, low-grade fuel and raw stocks that are sold.

Define our variables: 

* $x_{1}, ..., x_{5}$ are quantities of raw stock to produce high grade fuel.
* $x_{6}, ..., x_{10}$ are quantities of raw stock to produce low grade fuel.
* $x_{11}, ..., x_{15}$ are quantities of raw stock sold.
* $x_{16}$ are quantity of high grade fuel sold.
* $x_{17}$ are quantity of low grade fuel sold.

Regarding constraints:

The quantity of raw stock used to produce high grade fuel, low grade fuel, and sold, cannot exceed the quantity of raw stock available:

$$ x1 + x6 + x11  \leq 2000 $$
$$ x2 + x7 + x12  \leq 4000 $$
$$ x3 + x8 + x13  \leq 4000 $$
$$ x4 + x9 + x14  \leq 5000 $$
$$ x5 + x10 + x15 \leq 3000 $$

The quantity of raw stocks used to produce high grade fuel must match the quantity of high grade fuel produced and sold:

$$ x_1 + x_2 + x_3 + x_4 + x_5 = x_{16} $$ 

Similar for the quantity of low grade fuel produced and the quantity of raw stocks used to produce the low grade fuel: 

$$ x_6 + x_7 + x_8 + x_9 + x_{10} = x_{17} $$

We can blend raw stocks to create high grade fuel, and we want to ensure that the blended fuel grade is higher than 93. 

$$ 70x_1 + 80x_2 + 85x_3 + 90x_4 + 99x_5 \geq 93x_{16} $$

Similarly for low grad3e fuel: after blending raw stocks to create low grade fuel, we want to ensure that the blended fuel grade is higher than 85.

$$ 70x_6 + 80x_7 + 85x_8 + 90x_9 + 99x_{10} \geq 85x_{17} $$

The objective function is defined as:

$$ max\ \  9x_{11} + 12.5x_{12} + 12.5x_{13} + 27.5x_{14} + 27.5x_{15} + 37.5x_{16} + 28.5x_{17} $$


In [ ]:
using JuMP, GLPK

m = Model(GLPK.Optimizer)

#Declare Variables:
@variable(m, x1 >= 0)
@variable(m, x2 >= 0)
@variable(m, x3 >= 0)
@variable(m, x4 >= 0)
@variable(m, x5 >= 0)
@variable(m, x6 >= 0)
@variable(m, x7 >= 0)
@variable(m, x8 >= 0)
@variable(m, x9 >= 0)
@variable(m, x10 >= 0)
@variable(m, x11 >= 0)
@variable(m, x12 >= 0)
@variable(m, x13 >= 0)
@variable(m, x14 >= 0)
@variable(m, x15 >= 0)
@variable(m, x16 >= 0)
@variable(m, x17 >= 0)

# Add constraints
@constraint(m, constraint_1, x1 + x6 + x11  <= 2000)
@constraint(m, constraint_2, x2 + x7 + x12  <= 4000)
@constraint(m, constraint_3, x3 + x8 + x13  <= 4000)
@constraint(m, constraint_4, x4 + x9 + x14  <= 5000)
@constraint(m, constraint_5, x5 + x10 + x15 <= 3000)
@constraint(m, constraint_6, 70x1 + 80x2 + 85x3 + 90x4 + 99x5  >= 93x16)
@constraint(m, constraint_7, 70x6 + 80x7 + 85x8 + 90x9 + 99x10 >= 85x17)
@constraint(m, constraint_8, x1 + x2 + x3 + x4 + x5  == x16)
@constraint(m, constraint_9, x6 + x7 + x8 + x9 + x10 == x17)

#Set the objective
@objective(m, Max, 9x11 + 12.5x12 + 12.5x13 + 27.5x14 + 27.5x15 + 37.5x16 + 28.5x17)
print(m)


begin
    status = optimize!(m)
end
println("Objective value :", JuMP.objective_value(m))
#println("Optimal solution is x= \n", JuMP.value.(x))
println("x1 = ", JuMP.value(x1))  # => x1 = 1.0
println("x2 = ", JuMP.value(x2))
println("x3 = ", JuMP.value(x3))
println("x4 = ", JuMP.value(x4))
println("x5 = ", JuMP.value(x5))
println("x6 = ", JuMP.value(x6))
println("x7 = ", JuMP.value(x7))
println("x8 = ", JuMP.value(x8))
println("x9 = ", JuMP.value(x9))
println("x10 = ", JuMP.value(x10))
println("x11 = ", JuMP.value(x11))
println("x12 = ", JuMP.value(x12))
println("x13 = ", JuMP.value(x13))
println("x14 = ", JuMP.value(x14))
println("x15 = ", JuMP.value(x15))
println("x16 = ", JuMP.value(x16))
println("x17 = ", JuMP.value(x17))


### How many barrels of high and low grade fuel are produced and sold?  
2,125 barrels of high grade fuel are produced and sold. And 15,875 barrels of low grade fuel are produced and sold.

### How many barrels of each stock is not utilized in the production and instead is directly sold?  
No barrels of raw materials are sold. All of the materials are used in the production of fuel.

### What is the total revenue?
The total revenue is $532,125
### What is the contribution to revenue of each stock (1-5) and fuel sold (high and low)?

In [ ]:
2125 * 37.5

High grade fuel generates $79,687.50 in revenue

In [ ]:
15875 * 28.5

Low grade fuel generates $452,437.50 in revenue

In [ ]:
79687.5 + 452437.50

No raw materials are sold, so they do not contribute to the sales revenue